In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()
print('Spark Session Connected')

Spark Session Connected


24/01/11 18:39:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

df = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)
df.show()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [4]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [5]:
train,test = df.randomSplit([0.73,0.27])

als = ALS(maxIter=5,regParam=0.01,userCol='userId',ratingCol='rating',itemCol='movieId')





In [6]:
model = als.fit(train)

In [7]:
pred = model.transform(test)

In [8]:
pred.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      3|   1.0|    28|   1.7382914|
|      0|   1.0|    27|  0.22743125|
|      1|   1.0|    26| -0.23331816|
|      3|   1.0|    26|  0.19544902|
|      4|   1.0|    12|   1.5390917|
|      0|   1.0|    22|   1.7654026|
|      0|   1.0|    13|  0.28433222|
|      3|   1.0|    13|   1.5843201|
|      0|   1.0|     6|  0.48402742|
|      0|   1.0|     3|  -0.2798816|
|      2|   1.0|     3|   2.1646664|
|      4|   1.0|    19|  0.05237022|
|      1|   4.0|    15|   0.9972271|
|      4|   2.0|     8| -0.19653067|
|      2|   1.0|    23|   2.2715652|
|      1|   1.0|     7|  0.68270576|
|      2|   2.0|     7|   3.9259343|
|      4|   3.0|    10|-0.017412238|
|      4|   1.0|    24|    2.106141|
|      4|   1.0|    29|   1.4169595|
+-------+------+------+------------+
only showing top 20 rows



In [9]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [11]:
rmse = evaluator.evaluate(pred)
rmse

1.8675636823266404

In [12]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     12|    11|
|     18|    11|
|     19|    11|
|     21|    11|
|     22|    11|
|     23|    11|
|     32|    11|
|     35|    11|
|     36|    11|
|     39|    11|
|     43|    11|
|     47|    11|
|     48|    11|
|     51|    11|
|     72|    11|
|     75|    11|
|     76|    11|
|     94|    11|
+-------+------+



In [13]:
rec = model.transform(single_user)
rec.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     39|    11|  4.4875627|
|     35|    11|   4.409293|
|     23|    11|  4.0515895|
|     94|    11|  3.4875488|
|     19|    11|  3.0961645|
|     22|    11|  2.9057868|
|     12|    11|  2.7685382|
|     75|    11|  2.2943797|
|     72|    11|  2.2477846|
|     47|    11|   1.613253|
|     76|    11|  1.3896353|
|     48|    11|  1.3680104|
|     21|    11|  1.0656018|
|     18|    11|  0.9423659|
|     36|    11| 0.24349864|
|     32|    11|-0.33574936|
|     51|    11| -0.5673206|
|     43|    11|-0.76197964|
+-------+------+-----------+

